# Single Inference

> Warning: The major caveat here is that this really should be run via script and **NOT** in a notebook. That being said, this is the best way to introduce the concepts here.

In [1]:
import caffe
from larcv import larcv
import time

Welcome to ROOTaaS 6.06/08


The below instantiates the network based off the description in the prototxt and the saved pre-trained snapshot.

In [ ]:
prototxt = '/workspace/plainresnet10b/sp_plainresnet10b_test.prototxt'
model_snapshot = '/workspace/plainresnet10b/sp_plainresnet10b_iter_195750.caffemodel.h5'
net = caffe.Net(prototxt, model_snapshot,caffe.TEST)

Now, we need to setup the IO so that we can access the data inline with caffe. This will allow us to compare prediction with output.

In [ ]:
filler_name = 'DataFiller'
# check if larcv IO processor does in fact exist and registered in a factory
if not larcv.ThreadFillerFactory.exist_filler(filler_name):
    print '\033[93mFiller',filler_name,'does not exist...\033[00m'

# get IO instance, ThreadDatumFiller instance, from the factory
filler = larcv.ThreadFillerFactory.get_filler(filler_name)

# get # events to be processed 
num_events = filler.get_n_entries()
# force random access to be false for an inference
filler.set_random_access(False)

# construct our own IO to fetch ROI object for physics analysis, use RED mode w/ same input files
myio = larcv.IOManager(0,"AnaIO")
for f in filler.pd().io().file_list():
    myio.add_in_file(f)
myio.initialize()

## Running Inference

Here is the inference step. We're going to tell caffe to process the next batch of images, and we're going to setup some variables to point at the relevant data.

In [ ]:
# force the filler to move the next event-to-read pointer to the entry of our interest
filler.set_next_index(event_counter)

num_entries=10
net.forward()

while filler.thread_running():
    time.sleep(0.001)

roi_producer = filler.producer(1)

# get a vector of integers that record TTree entry numbers processed in this mini-batch
entries = filler.processed_entries()
if entries.size() != self._batch_size:
    print "\033[93mBatch counter mis-match!\033[00m"
    raise Exception

# retrieve data already read-and-stored-in-memory from caffe blob
adcimgs = net.blobs["data"].data    # this is image
labels  = net.blobs["label"].data   # this is label
scores  = net.blobs["softmax"].data # this is final output softmax vector
            

## Processing the output

For this next part, I'm not doing much in the way of processing. Usually, we'll save some results to CSV to compare later. For now, this is just print the information as it's coming out of the mini-batch.

In [ ]:
# loop over entry of mini-batch outcome
for index in xrange(num_entries):
    print "Entry: ", entries[index]
    print "npx: ", (adcimg > 0).sum()
    print "Label: ", int(label)
    print "Prediction: ", score.argmax()
    print "EMinus: ", score[0]
    print "Gamma: ", score[1]
    print "MuMinus: ", score[2]
    print "PiMinus: ", score[3]
    print "Proton: ", score[4]

    if int(label) == score.argmax():
        print "FOUND CORRECT PARTICLE"
                
    # now get ROI data from myroi, our separate IO handle, to record physics parameters
    myio.read_entry(entries[index])
    event_roi = myio.get_data(1,roi_producer)
                
    # loop over ROIs
    for roi in event_roi.ROIArray():
        if roi.MCSTIndex() == larcv.kINVALID_USHORT:
            print "Energy Start: ", roi.EnergyInit(), ": ", roi.EnergyDeposity
            print "Mass: ", larcv.ParticleMass(roi.PdgCode())
            print "Momentum At Start: ", (roi.Px(),roi.Py(),roi.Pz())
            print "Decay?: ", (np.abs(roi.PdgCode()) == 13 and np.abs(roi.ParentPdgCode()) == 211 or sv_vals['ndecay'] += 1
                        elif np.abs(roi.PdgCode()) == 11 and np.abs(roi.ParentPdgCode()) == 13)


## CLEANUP

More important than anything else.

In [ ]:
myio.finalize()
# destroy thread filler via factory, an owner
larcv.ThreadFillerFactory.destroy_filler(filler_name)